<center>
<img src="https://supportvectors.ai/logo-poster-transparent.png" width=400px style="opacity:0.8">
</center>

In [1]:
%run supportvectors-common.ipynb


<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



# Combined Multi-Embedding Retrieval Pipeline

## Overview

This pipeline demonstrates a sophisticated retrieval system that combines multiple embedding approaches to achieve optimal search performance. The system uses:

1. **Matryoshka Embeddings** - For efficient initial retrieval with truncated dimensions
2. **ColBERT Embeddings** - For detailed semantic matching with late interaction
3. **SPLADE Embeddings** - For sparse lexical retrieval with term expansion
4. **Cross-Encoder Reranking** - For final precision ranking

The pipeline implements a multi-stage retrieval approach that leverages the strengths of each embedding type while maintaining computational efficiency.

## Pipeline Architecture

The retrieval pipeline follows this hierarchical approach:

1. **Stage 1**: Use 64-dimensional Matryoshka embeddings for initial broad retrieval (500 candidates)
2. **Stage 2**: Refine with full 768-dimensional Matryoshka embeddings (100 candidates)
3. **Stage 3**: Parallel SPLADE retrieval (100 candidates)
4. **Stage 4**: Merge and filter with ColBERT embeddings (50 candidates)
5. **Stage 5**: Final reranking with Cross-Encoder (20 final results)

## Setup and Imports

In [2]:
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from fastembed import LateInteractionTextEmbedding, SparseTextEmbedding
from qdrant_client import QdrantClient, models
from retrieval_funnel import config
from retrieval_funnel.hf_text_utils import get_train_test_lists, tuples_list_to_dataset

## 1. Data Preparation

First, we load the subject chunks from the three subjects (Biology, Physics, History) that are used in the matryoshka embeddings notebook.

In [3]:
# Load subject chunks (biology=0, physics=1, history=2)
_, test_data = get_train_test_lists(cfg=config)
test_dataset = tuples_list_to_dataset(test_data)

# Extract text chunks and their labels
text_chunks = test_dataset["text"]
labels = test_dataset["label"]

print(f"Loaded {len(text_chunks)} text chunks")
print(f"Subject distribution: Biology={sum(1 for label in labels if label == 0)}, "
      f"Physics={sum(1 for label in labels if label == 1)}, "
      f"History={sum(1 for label in labels if label == 2)}")

Loaded 5331 text chunks
Subject distribution: Biology=1264, Physics=1295, History=2772


## 2. Initialize Embedding Models

We initialize all embedding models that will be used in the pipeline.

In [4]:
# Device setup
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cuda' if torch.cuda.is_available() else device
print(f"Using device: {device}")


Using device: mps


In [5]:

# 1. Matryoshka Embeddings Model Full 768
matryoshka_model = SentenceTransformer("tomaarsen/mpnet-base-nli-matryoshka").to(device)

In [6]:

# 2. Matryoshka Embeddings Model Truncated to 64
matryoshka_64_model = SentenceTransformer("tomaarsen/mpnet-base-nli-matryoshka", truncate_dim=64).to(device)

In [7]:

# 3. ColBERT Embeddings Model
colbert_model = LateInteractionTextEmbedding("colbert-ir/colbertv2.0")

In [8]:

# 4. SPLADE Embeddings Model
splade_model = SparseTextEmbedding("prithivida/Splade_PP_en_v1")

In [9]:

# 5. Cross-Encoder for final reranking
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2").to(device)

## 3. Initialize Qdrant Vector Store

We create a single collection with multiple vector types to enable efficient hybrid retrieval, following the approach from the [Qdrant reranking tutorial](https://qdrant.tech/documentation/advanced-tutorials/reranking-hybrid-search/).

In [10]:
# Initialize Qdrant client
qdrant_client = QdrantClient("localhost", port=6333) #QdrantClient(path="./qdrant_db")

# Single collection name for all embeddings
HYBRID_COLLECTION = "hybrid_search"

# Create a single collection with multiple vector types
try:
    qdrant_client.recreate_collection(
        collection_name=HYBRID_COLLECTION,
        vectors_config={
            "matryoshka_64": models.VectorParams(
                size=64,
                distance=models.Distance.COSINE,
            ),
            "matryoshka_768": models.VectorParams(
                size=768,
                distance=models.Distance.COSINE,
            ),
            "colbert": models.VectorParams(
                size=128,
                distance=models.Distance.COSINE,
                multivector_config=models.MultiVectorConfig(
                    comparator=models.MultiVectorComparator.MAX_SIM,
                ),
                hnsw_config=models.HnswConfigDiff(m=0)  # Disable HNSW for reranking
            ),
        },
        sparse_vectors_config={
            "splade": models.SparseVectorParams(
                modifier=models.Modifier.IDF
            )
        }
    )
    print(f"Created hybrid collection: {HYBRID_COLLECTION}")
except Exception as e:
    print(f"Collection {HYBRID_COLLECTION} already exists or error: {e}")

Created hybrid collection: hybrid_search


## 4. Generate and Store Embeddings

We generate embeddings for all text chunks using each model and store them in the hybrid collection.

In [11]:
def store_embeddings_in_qdrant():
    """Generate and store all embeddings in a single hybrid collection"""
    
    # Batch size for processing
    batch_size = 16
    
    for i in range(0, len(text_chunks), batch_size):
        batch_texts = text_chunks[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        
        print(f"Processing batch {i//batch_size + 1}/{(len(text_chunks) + batch_size - 1)//batch_size}")
        
        # 1. Generate Matryoshka embeddings (64 and 768 dimensions)
        matryoshka_64_embeddings = matryoshka_64_model.encode(
            batch_texts, 
            convert_to_numpy=True
        )
        
        matryoshka_768_embeddings = matryoshka_model.encode(
            batch_texts, 
            convert_to_numpy=True
        )
        
        # 2. Generate ColBERT embeddings
        colbert_embeddings = list(colbert_model.embed(batch_texts))
        
        # 3. Generate SPLADE embeddings
        splade_embeddings = list(splade_model.embed(batch_texts))
        
        # Store in single hybrid collection
        points = []
        
        for j, (text, label) in enumerate(zip(batch_texts, batch_labels)):
            doc_id = i + j
            
            # Prepare payload
            payload = {
                "id": doc_id,
                "text": text,
                "label": int(label),
                "subject": ["Biology", "Physics", "History"][int(label)]
            }
            
            # Create point with all vector types
            point = models.PointStruct(
                id=doc_id,
                payload=payload,
                vector={
                    "matryoshka_64": matryoshka_64_embeddings[j].tolist(),
                    "matryoshka_768": matryoshka_768_embeddings[j].tolist(),
                    "colbert": colbert_embeddings[j],
                    "splade": splade_embeddings[j].as_object()
                }
            )
            points.append(point)
        
        # Upload to single collection
        qdrant_client.upload_points(HYBRID_COLLECTION, points)
    
    print("All embeddings stored successfully in hybrid collection!")

### Execute the embedding generation and storage

In [12]:
store_embeddings_in_qdrant()

Processing batch 1/334
Processing batch 2/334
Processing batch 3/334
Processing batch 4/334
Processing batch 5/334
Processing batch 6/334
Processing batch 7/334
Processing batch 8/334
Processing batch 9/334
Processing batch 10/334
Processing batch 11/334
Processing batch 12/334
Processing batch 13/334
Processing batch 14/334
Processing batch 15/334
Processing batch 16/334
Processing batch 17/334
Processing batch 18/334
Processing batch 19/334
Processing batch 20/334
Processing batch 21/334
Processing batch 22/334
Processing batch 23/334
Processing batch 24/334
Processing batch 25/334
Processing batch 26/334
Processing batch 27/334
Processing batch 28/334
Processing batch 29/334
Processing batch 30/334
Processing batch 31/334
Processing batch 32/334
Processing batch 33/334
Processing batch 34/334
Processing batch 35/334
Processing batch 36/334
Processing batch 37/334
Processing batch 38/334
Processing batch 39/334
Processing batch 40/334
Processing batch 41/334
Processing batch 42/334
P

## 5. Multi-Stage Retrieval Pipeline

Now we implement the retrieval pipeline that combines all embedding approaches using the `prefetch` feature that Qdrant provides.

In [15]:
class MultiEmbeddingRetrievalPipelineQdrant:
    """Multi-stage retrieval pipeline combining Matryoshka, ColBERT, SPLADE, and Cross-Encoder using sequential refinement"""
    
    def __init__(self, qdrant_client, matryoshka_model, matryoshka_64_model, colbert_model, splade_model, cross_encoder):
        self.qdrant_client: QdrantClient = qdrant_client
        self.matryoshka_model = matryoshka_model
        self.matryoshka_64_model = matryoshka_64_model
        self.colbert_model = colbert_model
        self.splade_model = splade_model
        self.cross_encoder = cross_encoder
        
        # Single collection name
        self.hybrid_collection = HYBRID_COLLECTION

    def hybrid_search(self, query: str, 
                      matryoshka_64_limit: int = 500, 
                      matryoshka_768_limit: int = 100, 
                      splade_limit: int = 100, 
                      colbert_limit: int = 50):
        """Hybrid search pipeline in Qdrant

        Args:
            query (str): The query to search for
            matryoshka_64_limit (int, optional): The number of candidates to retrieve from Matryoshka 64D. Defaults to 500.
            matryoshka_768_limit (int, optional): The number of candidates to retrieve from Matryoshka 768D. Defaults to 100.
            splade_limit (int, optional): The number of candidates to retrieve from SPLADE. Defaults to 100.
            colbert_limit (int, optional): The number of candidates to retrieve from ColBERT. Defaults to 50.

        Returns:
            list: The list of candidates
        """
        matryoshka_64_vectors = self.matryoshka_64_model.encode([query], convert_to_numpy=True)[0]
        matryoshka_768_vectors = self.matryoshka_model.encode([query], convert_to_numpy=True)[0]
        colbert_vectors = list(self.colbert_model.query_embed(query))[0]
        splade_vectors = list(self.splade_model.embed([query]))[0]

        prefetch_matryoshka_64 = models.Prefetch(
            query=matryoshka_64_vectors.tolist(),
            using="matryoshka_64",
            limit=matryoshka_64_limit,
        )
        prefetch_matryoshka_768 = models.Prefetch(
            query=matryoshka_768_vectors.tolist(),
            prefetch=prefetch_matryoshka_64,
            using="matryoshka_768",
            limit=matryoshka_768_limit,
        )
        prefetch_splade = models.Prefetch(
            query=models.SparseVector(**splade_vectors.as_object()),
            using="splade",
            limit=splade_limit,
        )
        prefetch_merged = [prefetch_matryoshka_768, prefetch_splade]

        response = self.qdrant_client.query_points(
            collection_name=self.hybrid_collection,
            prefetch=prefetch_merged,
            query=colbert_vectors,
            using="colbert",
            with_payload=True,
            limit=colbert_limit,
        )
        print(f"Retrieved {len(response.points)} candidates from the HYBRID search pipeline")
        return response.points          
    
    def cross_encoder_reranking(self, query: str, candidates, cross_encoder_limit: int = 20):
        """Reranking using Cross-Encoder"""
        print(f"Final reranking to {cross_encoder_limit} results with Cross-Encoder...")
        
        # Prepare query-document pairs for cross-encoder
        query_doc_pairs = []
        for result in candidates:
            query_doc_pairs.append([query, result.payload["text"]])
        
        # Get cross-encoder scores
        cross_encoder_scores = self.cross_encoder.predict(query_doc_pairs)
        
        # Combine scores with results
        scored_results = []
        for i, result in enumerate(candidates):
            scored_results.append({
                "id": result.id,
                "payload": result.payload,
                "score": float(cross_encoder_scores[i]),
                "distance": result.score
            })
        
        # Sort by cross-encoder score (higher is better)
        scored_results.sort(key=lambda x: x["score"], reverse=True)
        
        # Return top results
        final_results = scored_results[:cross_encoder_limit]
        
        print(f"Cross-Encoder reranking complete: {len(final_results)} results")
        return final_results
    
    def retrieve(self, query: str, use_cross_encoder: bool = True):
        """Execute the complete multi-stage retrieval pipeline"""
        print(f"\n=== Starting Multi-Stage Retrieval for Query: '{query}' ===\n")

        colbert_results = self.hybrid_search(query)
        
        if use_cross_encoder:
            # Final reranking with Cross-Encoder
            final_results = self.cross_encoder_reranking(query, colbert_results, cross_encoder_limit=20)
        else:
            # Use ColBERT results directly
            final_results = [
                {
                    "id": result.id,
                    "payload": result.payload,
                    "score": result.score,
                    "distance": result.score
                }
                for result in colbert_results[:20]
            ]
        
        print(f"\n=== Retrieval Complete: {len(final_results)} final results ===\n")
        return final_results

# Initialize the pipeline
pipeline_qdrant = MultiEmbeddingRetrievalPipelineQdrant(
    qdrant_client=qdrant_client,
    matryoshka_model=matryoshka_model,
    matryoshka_64_model=matryoshka_64_model,
    colbert_model=colbert_model,
    splade_model=splade_model,
    cross_encoder=cross_encoder
)

## 6. Testing the Pipeline

Let's test the pipeline with various queries to demonstrate its effectiveness.

In [25]:
def test_pipeline_qdrant(query: str):
    """Test the multi-embedding retrieval pipeline with a query"""
    
    print(f"\n{'='*80}")
    print(f"TEST QUERY: {query}")
    print(f"{'='*80}")
    
    # Execute retrieval
    results = pipeline_qdrant.retrieve(query)
    
    # Display top results
    print("\nTOP RESULT:")
    print("-" * 60)
    print(f"Score: {results[0]['score']:.4f} | Subject: {results[0]['payload']['subject']}")
    print(f"   Text: {results[0]['payload']['text'][:150]}...")
    print()


In [26]:
# Test queries covering different subjects
test_queries = [
    "What is the relationship between force and acceleration?",
    "How do vaccines work in the human body?",
    "What were the major events of World War II?",
    "Explain the concept of velocity in physics",
    "What is the role of DNA in genetics?",
    "Who were the key figures in the American Revolution?"
]

In [28]:
# Run the test
for query in test_queries:
    test_pipeline_qdrant(query)


TEST QUERY: What is the relationship between force and acceleration?

=== Starting Multi-Stage Retrieval for Query: 'What is the relationship between force and acceleration?' ===

Retrieved 50 candidates from the HYBRID search pipeline
Final reranking to 20 results with Cross-Encoder...
Cross-Encoder reranking complete: 20 results

=== Retrieval Complete: 20 final results ===


TOP RESULT:
------------------------------------------------------------
Score: 0.9102 | Subject: Physics
   Text: – force = mass ×acceleration. But now the ‘force’ is something you calculate, as the
vector sum of the forces acting on all the separate particles , a...


TEST QUERY: How do vaccines work in the human body?

=== Starting Multi-Stage Retrieval for Query: 'How do vaccines work in the human body?' ===

Retrieved 50 candidates from the HYBRID search pipeline
Final reranking to 20 results with Cross-Encoder...
Cross-Encoder reranking complete: 20 results

=== Retrieval Complete: 20 final results ===




## 7. Performance Analysis

Let's analyze the performance characteristics of each stage in the pipeline.

In [29]:
def analyze_pipeline_performance_qdrant():
    """Analyze the performance characteristics of the pipeline stages"""
    
    import time
    
    test_query = "What is the relationship between force and acceleration?"
    
    print("=== Pipeline Performance Analysis ===\n")
    
    # Stage 1 timing
    start_time = time.time()
    stage1_results = pipeline_qdrant.hybrid_search(test_query)
    stage1_time = time.time() - start_time
    
    # Stage 5 timing
    start_time = time.time()
    final_results = pipeline_qdrant.cross_encoder_reranking(test_query, stage1_results)
    stage2_time = time.time() - start_time
    
    # Print performance summary
    print(f"Stage 1 (Hybrid Search): {stage1_time:.3f}s - {len(stage1_results)} candidates")
    print(f"Stage 2 (Cross-Encoder): {stage2_time:.3f}s - {len(final_results)} final results")
    
    total_time = stage1_time + stage2_time
    print(f"\nTotal Pipeline Time: {total_time:.3f}s")
    
    # Subject distribution analysis
    subject_counts = {}
    for result in final_results:
        subject = result['payload']['subject']
        subject_counts[subject] = subject_counts.get(subject, 0) + 1
    
    print("\nFinal Results Subject Distribution:")
    for subject, count in subject_counts.items():
        print(f"  {subject}: {count} results")

# Run performance analysis
analyze_pipeline_performance_qdrant()

=== Pipeline Performance Analysis ===

Retrieved 50 candidates from the HYBRID search pipeline
Final reranking to 20 results with Cross-Encoder...
Cross-Encoder reranking complete: 20 results
Stage 1 (Hybrid Search): 0.085s - 50 candidates
Stage 2 (Cross-Encoder): 0.163s - 20 final results

Total Pipeline Time: 0.248s

Final Results Subject Distribution:
  Physics: 20 results
